## ROSES Optimal Interpolation (Kriging) Laboratory 
### 08/25/2020 Tony Lowry

In this simple lab you'll go through a few steps (with example python algorithms) for interpolation of fields and "Bayesian" likelihood filtering of H-kappa amplitude stacks of receiver functions from the IRIS EARS database. 

H-kappa stacks are a parameter-space representation of receiver function amplitudes at times predicted for Moho converted-phase arrivals (including reverberations) sampled at arrival times predicted as a function of crustal thickness (H) and seismic velocity ratio (kappa)...

The starting data set (in NV_llhks.txt) are ascii outputs from a Bayesian inversion of sites from Nevada (longitude, latitude, crustal thickness (km), Vp/Vs, station name).

Because the data are in lon-lat positions, our first step will be to project spherical coordinates onto an x-y (km) format suitable for use with PyKrige. We'll do this with GMT's [mapproject](https://gmt.soest.hawaii.edu/doc/latest/mapproject.html). 

The scale of the data set is small enough that the type of projection chosen will not matter much. However, if one uses the data for gravity modeling purposes, it is good practice to use an equal-area projection such as [Albers](https://en.wikipedia.org/wiki/Albers_projection): -Jblon0/lat0/lat1/lat2/scale; for Nevada, a reasonable projection would be 
-Jb-117/38.5/35/42/1. 
    
Whatever you choose though, be sure to specify a 1:1 scale (the 1 at the end of -J!), a range -R that includes all of the data, and -Fk to force distances in km-- Otherwise the variogram distances will be incorrect in the PyKrige scripts we're using!

The [PyKrige](https://pypi.org/project/PyKrige/) routine expects a file named NV_xyhks.txt, so name your output that...

In [ ]:
! gmt mapproject NV_llhks.txt -Jb-117/39/35/42/1 -R-120/-114/35/42 -Fk > NV_xyhks.txt

Now, being a somewhat novice python coder, I'm not sure how to read a datafile into python that includes a mix of float and text information... So, let's use awk to create a purely numerical data file without the site names. (We'll need that info later though so keep the original file!)

In [ ]:
! awk '{print  $1,$2,$3,$4}' NV_xyhks.txt > NV_xyhk.txt

## Part I: Create interpolated grids of the data and their uncertainties

The first step toward evaluating our data is to see what it looks like in map view, along with estimates of uncertainty:

In [ ]:
"""
Ordinary Kriging Example (Crustal thickness)
============================================

First read in the text data file; create the interpolation grid

"""

import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

with open('NV_xyhk.txt', 'r') as f:
  data = np.loadtxt(f)

gridx = np.arange(0.0, 540.0, 10.0)
gridy = np.arange(0.0, 780.0, 10.0)


PyKrige enables you to estimate a variogram model directly from the data or to specify parameters of a variogram model based on other information. (One can also plot the variogram model estimates, although here we've set that option to "False".) The "spherical" model chosen here is generally a good approximation for the behavior of these data, although it is worth being aware that the 89 sites represented in our data file is an extremely small number with which to evaluate a statistical model! Here the verbose setting enables us to see the (sill, nugget, range) parameters associated with the resulting variogram model parameterization.

In [ ]:
###############################################################################
# Create the ordinary kriging object. Required inputs are the X-coordinates of
# the data points, the Y-coordinates of the data points, and the Z-values of the
# data points. If no variogram model is specified, defaults to a linear variogram
# model. If no variogram model parameters are specified, then the code automatically
# calculates the parameters by fitting the variogram model to the binned
# experimental semivariogram. The verbose kwarg controls code talk-back, and
# the enable_plotting kwarg controls the display of the semivariogram.

# For crustal thickness H:

OK = OrdinaryKriging(
    data[:, 0],
    data[:, 1],
    data[:, 2],
    variogram_model="spherical",
    verbose=True,
    enable_plotting=False,
)

The execute function in PyKrige uses the variogram to then create the kriging matrix and evaluate the expected value (z, below) and variance (ss) at each interpolation point.

In [ ]:
###############################################################################
# Creates the kriged grid and the variance grid. Allows for kriging on a rectangular
# grid of points, on a masked rectangular grid of points, or with arbitrary points.
# (See OrdinaryKriging.__doc__ for more information.)

z, ss = OK.execute("grid", gridx, gridy)

In [ ]:
###############################################################################
# Writes the kriged grids of crustal thickness to an ASCII grid file;
#  plots both the one-sigma uncertainty and thickness grids.

kt.write_asc_grid(gridx, gridy, z, filename="OI_H.xyh")
plt.figure(1)
plt.imshow(np.sqrt(ss), origin='lower', cmap='rainbow')
plt.colorbar()
plt.ioff()
plt.show()
plt.figure(2)
plt.imshow(z, origin='lower', cmap='rainbow')
plt.colorbar()
plt.ioff()
plt.show()


Here, the uncertainty grid has been plotted first, with the crustal thickness below. 

Oh my! One of these sites is not like the others... And that's because your Nevada data file included one site, TA.M11A, with the crustal thickness value from the raw EARS H-K amplitude stack... The rest had been run through a joint Bayesian inversion using gravity and OI-spatial statistics, described in Lowry & Pérez-Gussinyé (Nature 2011) and Ma & Lowry (Tectonics, 2017).

![title](Raw.TA.M11A.png)

This site had maximum stack amplitude at a depth of 59 km-- A highly suspect Moho depth for the extensional Basin & Range province, where other measurements are uniformly in the range of about 25-40 km.

Note that the uncertainty shows no obvious evidence of the problem (the outlier value does influence the variogram, but that spreads the associated variance uniformly across the region and "assumes" this site is different because the regionalized variable itself is different there...)

In [ ]:
# For Vp/Vs, K:

OK = OrdinaryKriging(
    data[:, 0],
    data[:, 1],
    data[:, 3],
    variogram_model="spherical",
    verbose=True,
    enable_plotting=False,
)

In [ ]:
###############################################################################
# Creates the kriged grid and the variance grid. Allows for kriging on a rectangular
# grid of points, on a masked rectangular grid of points, or with arbitrary points.
# (See OrdinaryKriging.__doc__ for more information.)

zk, sk = OK.execute("grid", gridx, gridy)

In [ ]:
###############################################################################
# Writes the kriged grid to an ASCII grid file and plot it.

kt.write_asc_grid(gridx, gridy, z, filename="OI_H.xyh")
plt.figure(3)
plt.imshow(np.sqrt(sk), origin='lower', cmap='rainbow')
plt.colorbar()
plt.ioff()
plt.show()
plt.figure()
plt.imshow(zk, origin='lower', cmap='rainbow')
plt.colorbar()
plt.ioff()
plt.show()

For the Vp/Vs variation, the estimate is similarly anomalous at TA.M11A.

Time to apply an OI Bayesian inversion!

# Part II: OI probability density function parameters (at TA.M11A)

First we need to create a new data file that leaves out the suspect site; we'll do this using grep:

In [ ]:
! grep -v TA.M11A NV_xyhks.txt > NVsub_xyhks.txt

And again, let's strip out the site names so we can read the file into PyKrige:

In [ ]:
! awk '{print $1,$2,$3,$4}' NVsub_xyhks.txt > NVsub_xyhk.txt

In this instance, we don't want to make a gridded map but instead evaluate the expected value and variance for H-K at the TA.M11A site location, using all of the sites *except* TA.M11A. 

In [ ]:
! grep TA.M11A NV_xyhks.txt

Note that if you used different mapproject flags to get your x-y projection, your (x, y) position will be different than the one used below...

In [ ]:
"""
Ordinary Kriging Example (Crustal thickness)
============================================

First read in the text data file; Specify the interpolation site (at TA.M11A)

"""

import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

with open('NVsub_xyhk.txt', 'r') as f:
  data = np.loadtxt(f)

# This uses the x, y location of TA.M11A from a specific Albers equal area projection-- 
#   If this is not the x-y position in NV_xyhks.txt of the site you want to evaluate
#   the OI probability density function for, substitute the correct coordinates!

xs = 374.791883154
ys = 715.464502987

In [ ]:
###############################################################################
# Create the ordinary kriging object. Required inputs are the X-coordinates of
# the data points, the Y-coordinates of the data points, and the Z-values of the
# data points. If no variogram model is specified, defaults to a linear variogram
# model. If no variogram model parameters are specified, then the code automatically
# calculates the parameters by fitting the variogram model to the binned
# experimental semivariogram. The verbose kwarg controls code talk-back, and
# the enable_plotting kwarg controls the display of the semivariogram.

# For crustal thickness H:

OK = OrdinaryKriging(
    data[:, 0],
    data[:, 1],
    data[:, 2],
    variogram_model="spherical",
    verbose=True,
    enable_plotting=False,
)

In [ ]:
###############################################################################
# Evaluates the kriged expected value and sigma. 
# (See OrdinaryKriging.__doc__ for more information.)

zh, sh = OK.execute("points", xs, ys)

In [ ]:
###############################################################################
# Writes the pdf parameters:

print(zh,np.sqrt(sh))

Wait... What? Oh no! These estimates are non-physical!

Well, welcome to the world of kriging... Where some combinations of variogram parameters and site distributions can result in matrices that are ill-conditioned or even singular... Resulting in estimates of expected-values and variance that are nonsensical. In [our Fortran inversion code package](http://aconcagua.geol.usu.edu/~arlowry/code_release.html), we use Singular Value Decomposition to mitigate this problem. (I'm not sure what PyKrige uses for matrix inversion, but SVD slows things down. A lot.)

Still... Knowing this, we might be able to get a more numerically stable result by using a more robust variogram model. Here I'm specifying the variogram derived from our Fortran-platform analysis, incorporating more than 5000 sites across North America:

In [ ]:
###############################################################################
# Create the ordinary kriging object. Required inputs are the X-coordinates of
# the data points, the Y-coordinates of the data points, and the Z-values of the
# data points. If no variogram model is specified, defaults to a linear variogram
# model. If no variogram model parameters are specified, then the code automatically
# calculates the parameters by fitting the variogram model to the binned
# experimental semivariogram. The verbose kwarg controls code talk-back, and
# the enable_plotting kwarg controls the display of the semivariogram.

# For crustal thickness H: 
# Sill is global variance; Range is decorrelation lengthscale; Nugget is zero-lag variance

OK = OrdinaryKriging(
    data[:, 0],
    data[:, 1],
    data[:, 2],
    variogram_model="spherical",
    variogram_parameters=[83.284, 1560, 7.740],
    verbose=True,
    enable_plotting=False,
)

In [ ]:
###############################################################################
# Evaluates the kriged expected value and sigma. 
# (See OrdinaryKriging.__doc__ for more information.)

zh, sh = OK.execute("points", xs, ys)

In [ ]:
###############################################################################
# Writes the pdf parameters:

print(zh,np.sqrt(sh))

Whew! Success!

To be on the safe side, we'll specify a better-informed variogram model for Vp/Vs as well:

In [ ]:
# For Vp/Vs, K:

OK = OrdinaryKriging(
    data[:, 0],
    data[:, 1],
    data[:, 3],
    variogram_model="spherical",
    variogram_parameters=[0.00205, 600, 0.000723],
    verbose=False,
    enable_plotting=False,
)

In [ ]:
###############################################################################
# Evaluates the kriged expected value and sigma.
# (See OrdinaryKriging.__doc__ for more information.)

zk, sk = OK.execute("points", xs, ys)

In [ ]:
###############################################################################
# Writes the pdf parameters to screen:

print(zk,np.sqrt(sk))

Ah! Now we have all of the parameters we need to create a two-dimensional (multivariate) Gaussian probability density function suitable for use as a Likelihood function in Bayesian inversion. 

If I were a better coder I'd include python code for that, but instead I'm including a fortran code that you folks can compile and run (or use as a template for developing your own python script) if you so wish. And here are the results:

![title](OIF.png)

Happy kriging!